# GANomaly Notebook Results

## Initial Configurations

### Libraries import

In [ ]:
import os
import sys
import random
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import umap
from sklearn.decomposition import PCA
import cv2
from IPython.display import display, clear_output
from ipywidgets import interact, IntSlider

sys.path.append("../../")

In [ ]:
from utils.metrics import dagostinoPearson_test, andersonDarling_test, shapiroWilks_test, chiSquare_test, fOneWay_test
from utils.metrics import brownForsythe_test, levene_test, bartlett_test
from utils.metrics import mannWhitney_test, kruskalWallis_test, kolmogorovSmirnov_test
from utils.savers import generate_qq_plot

### Experiment selection

In [ ]:
experiment_id = "0009"
root_path = "/home/jefelitman/Saved_Models/Anomaly_parkinson/"
for i in sorted(os.listdir("/home/jefelitman/Saved_Models/Anomaly_parkinson/")):
    if experiment_id in i:
        experiment_folder = os.path.join(root_path, i)
experiment_folder

## Quantitative metrics

### Metrics loading

In [ ]:
train_metrics = pd.read_csv(os.path.join(experiment_folder, "metrics/train.csv"))
data = train_metrics
np.max(data, axis=0), np.min(data, axis=0), data.tail()

In [ ]:
test_metrics = pd.read_csv(os.path.join(experiment_folder, "metrics/test.csv"))
data = test_metrics
np.max(data, axis=0), np.min(data, axis=0), data.tail()

### Train Graphics

In [ ]:
path = os.path.join(experiment_folder, "outputs/graphics/quantitative/")
portions = 500
rows = 12
columns = 5
for metric in ["gen_error", "disc_error", "accuracy", "specificity"]:
    fig, axs = plt.subplots(rows, columns, figsize=(30, 25))
    fig.suptitle('{} over Epochs'.format(metric))
    for i in range(rows):
        for j in range(columns):
            axs[i, j].plot(train_metrics[metric][(i*columns+j)*portions:(i*columns+j+1)*portions])
            filename = 'train_{}.png'.format(metric)
    fig.savefig(path+filename)
    plt.close(fig)

### Test graphics

In [ ]:
path = os.path.join(experiment_folder, "outputs/graphics/quantitative/")
portions = 500
rows = 12
columns = 5
for metric in test_metrics.columns[1:]:
    fig, axs = plt.subplots(rows, columns, figsize=(30, 25))
    fig.suptitle('{} over Epochs'.format(metric))
    for i in range(rows):
        for j in range(columns):
            axs[i, j].plot(test_metrics[metric][(i*columns+j)*portions:(i*columns+j+1)*portions])
            filename = 'test_{}.png'.format(metric)
    fig.savefig(path+filename)
    plt.close(fig)

## Qualitative metrics

### Errors loading

In [ ]:
base_path = os.path.join(experiment_folder, "outputs/errors/")
for t in ["encoder", "contextual", "adversarial"]:
    for m in ["train", "test"]:
        classes = ["normal"] if m == "train" else ["normal", "abnormal"]
        for c in classes:
            all_data = "all_{}_{}".format(t, c)
            if all_data not in globals().keys():
                globals()[all_data] = np.r_[[]]
            
            errors = np.load(os.path.join(base_path, t, m, c+".npy"))
            globals()["{}_{}_{}".format(m, t, c)] = errors
            globals()[all_data] = np.concatenate([globals()[all_data], errors])

### Train means and stds

In [ ]:
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal"]:
        data = globals()["train_{}_{}".format(t,c)]
        print("{i} error ({j}): {m} +- {s}".format(
            i = t,
            j = c,
            m = np.mean(data),
            s = np.std(data)
        ))
    print("")

### Train statistical tests

In [ ]:
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal"]:
        print("-------------------------------------------------")
        data = globals()["train_{}_{}".format(t,c)]
        norm_dist = stats.norm.rvs(loc=0, scale=1, size=data.shape)
        print("D'agostino-Pearson test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = dagostinoPearson_test(data)
        ))
        print("Shapiro-Wilks test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = shapiroWilks_test(data)
        ))
        print("Barlett test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = bartlett_test(data, norm_dist)
        ))
        print("Levene test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = levene_test(data, norm_dist)
        ))
        print("F1 test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = fOneWay_test(data, norm_dist)
        ))
        print("-------------------------------------------------")
    print("")

### Test means and stds

In [ ]:
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal", "abnormal"]:
        data = globals()["test_{}_{}".format(t,c)]
        print("{i} error ({j}): {m} +- {s}".format(
            i = t,
            j = c,
            m = np.mean(data),
            s = np.std(data)
        ))
    print("")

### Test statistical tests

In [ ]:
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal", "abnormal"]:
        print("-------------------------------------------------")
        data = globals()["test_{}_{}".format(t,c)]
        norm_dist = stats.norm.rvs(loc=0, scale=1, size=data.shape)
        print("D'agostino-Pearson test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = dagostinoPearson_test(data)
        ))
        print("Shapiro-Wilks test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = shapiroWilks_test(data)
        ))
        print("Barlett test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = bartlett_test(data, norm_dist)
        ))
        print("Levene test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = levene_test(data, norm_dist)
        ))
        print("F1 test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = fOneWay_test(data, norm_dist)
        ))
        print("-------------------------------------------------")
    print("")

### All data means and stds

In [ ]:
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal", "abnormal"]:
        data = globals()["all_{}_{}".format(t,c)]
        print("{i} error ({j}): {m} +- {s}".format(
            i = t,
            j = c,
            m = np.mean(data),
            s = np.std(data)
        ))
    print("")

### All data statistical tests

In [ ]:
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal", "abnormal"]:
        print("-------------------------------------------------")
        data = globals()["all_{}_{}".format(t,c)]
        norm_dist = stats.norm.rvs(loc=0, scale=1, size=data.shape)
        print("D'agostino-Pearson test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = dagostinoPearson_test(data)
        ))
        print("Shapiro-Wilks test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = shapiroWilks_test(data)
        ))
        print("Barlett test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = bartlett_test(data, norm_dist)
        ))
        print("Levene test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = levene_test(data, norm_dist)
        ))
        print("F1 test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = fOneWay_test(data, norm_dist)
        ))
        print("-------------------------------------------------")
    print("")

### Train graphics

#### QQ plots

In [ ]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/")
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal"]:
        generate_qq_plot(globals()["train_{}_{}".format(t,c)], save_path, "train_{}_{}".format(t,c))

In [ ]:
import statsmodels.api as sm
from scipy import stats

def min_max_scaler(data, new_max, new_min):
    data_std = (data - np.min(data)) / (np.max(data) - np.min(data))
    data_scaled = data_std * (new_max - new_min) + new_min
    return data_scaled

In [ ]:
data = test_contextual_abnormal
stats.chisquare(data, ddof=data.shape[0]-2)

In [ ]:
data_1 = test_encoder_normal
data_2 = test_encoder_abnormal
stats.mannwhitneyu(data_1, data_2)

In [ ]:
stats.norm()

In [ ]:
data = train_encoder_normal
for i in range(1, data.shape[0]):
    sm.qqplot(min_max_scaler(data, 1, -1), dist=stats.chi2(i))
    ax = plt.gca()
    plt.plot(ax.get_xlim(), ax.get_ylim(), color="r")
    input("Degree of freedom {}".format(i))
    clear_output()
    plt.close()

### Test graphics

#### Box plots

In [ ]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/")
rows = 1
columns = 3
fig, axs = plt.subplots(rows, columns, figsize=(15, 10))
for i, t in enumerate(["encoder", "contextual", "adversarial"]):
    data = "test_{}_".format(t)
    axs[i].boxplot([globals()[data+"normal"], globals()[data+"abnormal"]], labels=['Normal', 'Abnormal'])
    axs[i].set_title("{} errors".format(t))

filename = 'test_boxplot.png'
fig.savefig(save_path+filename)
plt.close(fig)

#### QQ plots

In [ ]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/")
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal", "abnormal"]:
        generate_qq_plot(globals()["test_{}_{}".format(t,c)], save_path, "test_{}_{}".format(t,c))

### All data graphics

#### Box plots

In [ ]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/")
rows = 1
columns = 3
fig, axs = plt.subplots(rows, columns, figsize=(15, 10))
for i, t in enumerate(["encoder", "contextual", "adversarial"]):
    data = "all_{}_".format(t)
    axs[i].boxplot([globals()[data+"normal"], globals()[data+"abnormal"]], labels=['Normal', 'Abnormal'])
    axs[i].set_title("{} errors".format(t))

filename = 'all_data_boxplot.png'
fig.savefig(save_path+filename)
plt.close(fig)

#### QQ plots

In [ ]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/")
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal", "abnormal"]:
        generate_qq_plot(globals()["all_{}_{}".format(t,c)], save_path, "all_data_{}_{}".format(t,c))

## Visual results

### UMAP latent vectors

In [ ]:
base_path = os.path.join(experiment_folder, "outputs/latent_vectors/")
for t in ["in", "out"]:
    for m in ["train", "test"]:
        classes = ["normal"] if m == "train" else ["normal", "abnormal"]
        for c in classes:
            all_data = "all_{}_{}".format(t, c)
            if all_data not in globals().keys():
                globals()[all_data] = []
            
            data = "{}_{}_{}".format(m, t, c)
            globals()[data] = []
            path = os.path.join(base_path, t, m, c)
            for file in sorted(os.listdir(path)):
                vector = np.load(os.path.join(path, file))
                globals()[data].append(vector)
                globals()[all_data].append(vector)
            globals()[data] = np.r_[globals()[data]]
for t in ["in", "out"]:
    for c in ["normal", "abnormal"]:
        all_data = "all_{}_{}".format(t, c)
        globals()[all_data] = np.r_[globals()[all_data]]

In [ ]:
save_path = os.path.join(experiment_folder, "outputs/graphics/visuals/")
for t in ["in", "out"]:
    for m in ["train", "test", "all"]:
        classes = ["normal"] if m == "train" else ["normal", "abnormal"]
        globals()["{}_mapped_{}".format(m, t)] = umap.UMAP(random_state = 8128).fit_transform(np.concatenate(
                [globals()["{}_{}_{}".format(m, t, c)] for c in classes], axis=0
            )
        )
        
        divisor = globals()["{}_{}_normal".format(m, t)].shape[0]
        plt.scatter(globals()["{}_mapped_{}".format(m, t)][:divisor,0],
            globals()["{}_mapped_{}".format(m, t)][:divisor,1], color="blue", s=5, label="Normal"
        )
        if "abnormal" in classes:
            plt.scatter(globals()["{}_mapped_{}".format(m, t)][divisor:,0],
                globals()["{}_mapped_{}".format(m, t)][divisor:,1], color="red", s=5, label="Abnormal"
            )
        plt.legend()
        plt.title("UMAP for {}put data in {}".format(t, m))
        filename = '{}_umap_{}put.png'.format(m, t)
        plt.savefig(save_path+filename)
        plt.close()

### Video comparision

In [ ]:
def show_all_data(volumes):
    frame_slider = IntSlider(value=32, min=1, max=volumes[0].shape[0], step=1)
    volume_slider = IntSlider(min=1, max=len(volumes), step=1)

    def update_frame_max(*args):
        frame_slider.max = len(sorted(os.listdir(path)))
    volume_slider.observe(update_frame_max, 'value')

    interact(lambda volume, frame: plt.imshow(volumes[volume-1][frame-1,:,:,1], cmap="gray"), 
             volume=volume_slider, frame=frame_slider
    );
show_all_data(classes[0])

In [ ]:
for i in range(0, 88):
    video_index = i
    mode = "test"
    video_class = "abnormal"

    base_path = os.path.join(experiment_folder, "outputs/samples/")
    for t in ["real", "fake", "substraction"]:
        path = os.path.join(base_path, t, mode, video_class)
        video = sorted(os.listdir(path))[video_index]
        data = "video_{}".format(t)
        globals()[data] = []
        for frame in sorted(os.listdir(os.path.join(path, video))):
            globals()[data].append(cv2.cvtColor(
                cv2.imread(
                    os.path.join(path, video, frame)
                ), cv2.COLOR_BGR2RGB
            ))
        globals()[data] = np.r_[globals()[data]]  

    assert video_real.shape[0] == video_fake.shape[0] == video_substraction.shape[0]

    widget = IntSlider(value=32, min=1, max=video_real.shape[0], step=1)

    interact(lambda frame: plt.imshow(video_real[frame-1]), frame=widget)
    interact(lambda frame: plt.imshow(video_fake[frame-1]), frame=widget)
    interact(lambda frame: plt.imshow(video_substraction[frame-1]), frame=widget)
    input("indice actual: {}".format(i))
    clear_output()